In [1]:
import re
import os
import json
import codecs
import numpy as np
import pandas as pd
from collections import Counter

In [74]:
rawPath="E:/project/xc_KG/EandR/code/process/data/training.csv"
dataPath="./rawdata/standard.csv"
dictSaveDir='./rawdata'

In [80]:
def csvToStandard(rawPath, savePath="./rawdata/standard.csv", names=["head", "relation", "tail"], 、
                  header=0, sep=",", encoding="utf-8", compression="infer"):
    """数据清洗，从清洗原始数据得到标准数据"""
    print("INFO : Loading data of type %s" % os.path.splitext(rawPath)[-1])
    rawDf = pd.read_csv(rawPath,
                        sep=sep,
                        encoding=encoding,
                        names=names,
                        header=header,
                        keep_default_na=False,  # ==> Solve default nan
                        compression=compression,# ==> Solve ZIP and TAR
                        error_bad_lines=False,  # ==> Solve inconsistent lines
                        warn_bad_lines=False,   # ==> Solve inconsistent lines
                        skip_blank_lines=True)  # ==> Solve blank lines
    print(rawDf)
    print("INFO : Remove the space from the head and tail of entity.")
    rawDf = rawDf.applymap(lambda x: x.strip())  # Rid of the space in head and tail of entity
    print("INFO : Drop line with nan value.")    # Attention: " " should be removed.
    rawDf.replace({'': np.nan}, inplace=True)
    rawDf.dropna(axis=0, how='any', inplace=True)   #存在空格的行全部去掉

    print("INFO : Save standard data to file path : %s" % savePath)
    rawDf.to_csv(savePath, sep="\t", index=None, encoding="utf-8")  
    print("INFO : Successfully saving!")

def generateDict(dataPath, dictSaveDir):
    if type(dataPath) == str:
        print("INFO : Loading standard data!")
        rawDf = pd.read_csv(dataPath,
                            sep="\t", # 保存的时候用的就是/t，这里需要对应
                            header=0,
                            keep_default_na=False,
                            encoding="utf-8")
    elif type(dataPath) == list:
        print("INFO : Loading a list of standard data!")
        rawDf = pd.concat([pd.read_csv(p,
                                       sep="\t",
                                       keep_default_na=False,
                                       encoding="utf_8_sig") for p in dataPath], axis=0)   #列表生成式
        rawDf.reset_index(drop=True, inplace=True)
    # 对头尾实体和关系计数
    headCounter = Counter(rawDf["head"])
    tailCounter = Counter(rawDf["tail"])
    relaCounter = Counter(rawDf["relation"])

    # Generate entity and relation list
    entityList = list((headCounter + tailCounter).keys())
    relaList = list(relaCounter.keys())

    # Transform to index dict
    print("INFO : Transform to index dict")
    entityDict = dict([(word, ind) for ind, word in enumerate(entityList)])
    relaDict = dict([(word, ind) for ind, word in enumerate(relaList)])
    # Save path
    entityDictPath = os.path.join(dictSaveDir, "entityDict.json")
    relaDictPath = os.path.join(dictSaveDir, "relationDict.json")

    # Saving dicts
    json.dump({"stoi": entityDict, "itos": entityList}, open(entityDictPath, "w",encoding='utf-8'),、
              ensure_ascii=False)  #数据格式转为json并写入文件中
    json.dump({"stoi": relaDict, 'itos': relaList}, open(relaDictPath, "w",encoding='utf-8'),、
              ensure_ascii=False)

def splitData(dataPath, saveDir, evalPortion=0.1):
    """区分训练集和测试集"""
    assert evalPortion >= 0 and evalPortion <= 1.0
    print("INFO : Loading standard data!")
    rawDf = pd.read_csv(dataPath,
                        sep="\t",
                        header=None,
                        names=["head", "relation", "tail"],
                        keep_default_na=False,
                        encoding="utf-8")
    # Split eval data 划分训练集和测试集
    evalDf = rawDf.sample(frac=evalPortion)    #frac为抽样比例，返回为dataframe
    rawDf.drop(labels=evalDf.index, axis=0, inplace=True)
    evalDf.reset_index(drop=True, inplace=True)
    rawDf.reset_index(drop=True, inplace=True)

    # Save path
    trainPath = os.path.join(saveDir, "train.txt")
    validPath = os.path.join(saveDir, "valid.txt")
    rawDf.to_csv(trainPath, sep="\t", header=None, index=None, encoding="utf-8")
    evalDf.to_csv(validPath, sep="\t", header=None, index=None, encoding="utf-8")


In [85]:
# csvToStandard(rawPath)
generateDict(dataPath,dictSaveDir)


INFO : Loading standard data!
INFO : Transform to index dict


In [2]:
import math
from torch.utils.data import *

In [24]:
entityDictPath='./rawdata/entityDict.json'
relationDictPath='./rawdata/relationDict.json'
# posDataPath='./rawdata/standard.csv'
posDataPath='./rawdata/8yuedataC_id.csv'

In [9]:
class tripleDataset(Dataset):
    def __init__(self, entityDictPath, relationDictPath, posDataPath):  #posdatapath为训练集路径
        super(Dataset, self).__init__()
        # Load entity-index dict and relation-index dict
        print("INFO : Load entity and relation dict.")
        self.entityDict = json.load(open(entityDictPath, "r",encoding='utf-8'))["stoi"]     #{word:ind} of entity
        self.relationDict = json.load(open(relationDictPath, "r",encoding='utf-8'))["stoi"] #{word:ind} of relationship
        
        # Transform entity and relation to index
        print("INFO : Loading positive triples and transform to index.")
        self.posDf = pd.read_csv(posDataPath,                             #trainning data/ type:dataframe
                                 sep="\t",
#                                  names=["head", "relation", "tail"],
#                                  header=None,
                                 encoding="utf-8",
                                 keep_default_na=False)


        self.transformToIndex(self.posDf, repDict={"head":self.entityDict,
                                                   "relation":self.relationDict,
                                                   "tail":self.entityDict})
#         self.posDf.dropna()

    def generateNegSamples(self, repProba=0.5, exProba=0.5, repSeed=0, exSeed=0, headSeed=0, tailSeed=0):  #用于生成负样本
        assert repProba >= 0 and repProba <= 1.0 and exProba >= 0 and exProba <= 1.0
        # Generate negtive samples from positive samples
        print("INFO : Generate negtive samples from positive samples.")
        self.negDf = self.posDf.copy()          #此处copy正样本三元组进行负样本集的初始化
        np.random.seed(repSeed)
        repProbaDistribution = np.random.uniform(low=0.0, high=1.0, size=(len(self.negDf), ))
        np.random.seed(exSeed)
        exProbaDistribution = np.random.uniform(low=0.0, high=1.0, size=(len(self.negDf), ))
        shuffleHead = self.negDf["head"].sample(frac=1.0, random_state=headSeed)
        shuffleTail = self.negDf["tail"].sample(frac=1.0, random_state=tailSeed)  #打乱了顺序，头尾实体顺序打乱，关系不动，所以每行的三元组已经为反例

        # Replacing head or tail
        def replaceHead(relHead, shuffHead, shuffTail, repP, exP):
            if repP >= repProba:
                '''
                Not replacing head.self.negD
                '''
                return relHead
            else:
                if exP > exProba:
                    '''
                    Replacing head with shuffle head.
                    '''
                    return shuffHead  # 用打乱的头实体替换
                else:
                    '''
                    Replacing head with shuffle tail.
                    '''
                    return shuffTail # 用打乱的尾实体替换
        def replaceTail(relTail, shuffHead, shuffTail, repP, exP):
            if repP < repProba:
                '''
                Not replacing tail.
                '''
                return relTail
            else:
                if exP > exProba:
                    '''
                    Replacing tail with shuffle tail.
                    '''
                    return shuffTail
                else:
                    '''
                    Replacing head with shuffle head.
                    '''
                    return shuffHead

        # 得到了头尾实体仅替换掉一个的负样本集
        self.negDf["head"] = list(map(replaceHead, self.negDf["head"], \
                                      shuffleHead, shuffleTail, repProbaDistribution, exProbaDistribution))
        self.negDf["tail"] = list(map(replaceTail, self.negDf["tail"], \
                                      shuffleHead, shuffleTail, repProbaDistribution, exProbaDistribution))

  
    @staticmethod   #静态方法   将posDF中的元素转为entity、relation字典中的values
    def transformToIndex(csvData:pd.DataFrame, repDict:dict): #函数参数中的冒号是参数的类型建议符，告诉程序员希望传入的实参的类型
        for col in repDict.keys():    #key为head，relation,tail
            csvData[col] = csvData[col].apply(lambda x:repDict[col][x])

    def __len__(self):
        return len(self.posDf)

    def __getitem__(self, item):
        if hasattr(self, "negDf"):
            return np.array(self.posDf.iloc[item,:3]), np.array(self.negDf.iloc[item,:3])  #序列索引，返回3列   包含正样本和负样本的元组
        else:
            return np.array(self.posDf.iloc[item,:3])

In [25]:
dataset=tripleDataset(entityDictPath,relationDictPath,posDataPath)

INFO : Load entity and relation dict.
INFO : Loading positive triples and transform to index.


In [26]:
dataset.generateNegSamples()  #产生负样本
print(dataset.negDf)
# dataset[[1,100]]


INFO : Generate negtive samples from positive samples.
         head  relation  tail
0           0         0    36
1           1         1   235
2           0         1    53
3           2         1   158
4          20         2   136
...       ...       ...   ...
3697448   164      7993    93
3697449   215      7993    84
3697450    53      7990   195
3697451   256      7992   134
3697452   127      7992   133

[3697453 rows x 3 columns]


In [13]:
import torch
import codecs
import torch.nn as nn
import torch.nn.functional as F

In [14]:
class TransD(nn.Module):
    def __init__(self, entityNum, relationNum, entityDim, relationDim, margin=2.0, L=2):
        """确定实体和关系的数量和维数"""
        super(TransD, self).__init__() # nn.Module
        self.model = "TransD"
        self.margin = margin
        self.L = L

        # Initialize the entity and relation embedding and projection embedding
        self.entityEmbedding = nn.Embedding(num_embeddings=entityNum,
                                            embedding_dim=entityDim)
        self.entityMapEmbedding = nn.Embedding(num_embeddings=entityNum,
                                             embedding_dim=entityDim)
        self.relationEmbedding = nn.Embedding(num_embeddings=relationNum,
                                              embedding_dim=relationDim)
        self.relationMapEmbedding = nn.Embedding(num_embeddings=relationNum,
                                                 embedding_dim=relationDim)

        self.distfn = nn.PairwiseDistance(L)

    '''
    Calculate the score, steps are follows:
    Step1: Split input triples as head, relation and tail
    Step2: Calculate the mapping matrix Mrh and Mrt
    Step3: Calculate the mapping vector of head and tail
    Step4: Return the score
    '''
    def scoreOp(self, inputTriples):
        head, relation, tail = torch.chunk(inputTriples,
                                           chunks=3,
                                           dim=1)
        headp = torch.squeeze(self.entityMapEmbedding(head), dim=1)   # (B, 1, En) -> (B, En)
        head = torch.squeeze(self.entityEmbedding(head), dim=1)       # (B, 1, En) -> (B, En)
        tailp = torch.squeeze(self.entityMapEmbedding(tail), dim=1)   # (B, 1, En) -> (B, En)
        tail = torch.squeeze(self.entityEmbedding(tail), dim=1)       # (B, 1, En)  -> (B, En)
        relationp = torch.squeeze(self.relationMapEmbedding(relation), dim=1) # (B, 1, Em) -> (B, Em)
        relation = torch.squeeze(self.relationEmbedding(relation), dim=1)     # (B, 1, Em) -> (B, Em)

        '''
        relationp(B, Em) -> relationp(B, Em, 1)
        headp(B, En) -> headp(B, 1, En)
        tailp(B, En) -> tailp(B, 1, En)
        (B, Em, 1) * (B, 1, En) -> (B, Em, En)
        '''
        relatioDim = relation.size()[1] # [0]是批数据的大小 
        entityDim = head.size()[1]
        # 变换维度，增加一维
        relationp = torch.unsqueeze(relationp, dim=2)   # (B, Em, 1)
        headp = torch.unsqueeze(headp, dim=1)           # (B, 1, En)
        tailp = torch.unsqueeze(tailp, dim=1)           # (B, 1, En)
        
        if inputTriples.is_cuda: # 判断是否在GPU里
            Mrh = torch.matmul(relationp, headp) + torch.eye(relatioDim, entityDim).cuda(inputTriples.device.index)
            Mrt = torch.matmul(relationp, tailp) + torch.eye(relatioDim, entityDim).cuda(inputTriples.device.index)
        else:
            Mrh = torch.matmul(relationp, headp) + torch.eye(relatioDim, entityDim)
            Mrt = torch.matmul(relationp, tailp) + torch.eye(relatioDim, entityDim)
        # Map head and tail with mapping Matrix Mrh and Mrt
        # Mrh, Mrt : (B, Em, En)
        # head, tail : (B, En) -> (B, En, 1)
        # (B, Em, En) * (B, En, 1) -> (B, Em, 1)
        head = torch.unsqueeze(head, dim=2)
        tail = torch.unsqueeze(tail, dim=2)
        head = torch.squeeze(torch.matmul(Mrh, head), dim=2)   # (B, Em, 1) -> (B, Em)
        tail = torch.squeeze(torch.matmul(Mrt, tail), dim=2)   # (B, Em, 1) -> (B, Em)
        output = self.distfn(head+relation, tail)
        return output

    '''
    Normalize embedding
    '''
    def normalizeEmbedding(self):
        self.entityEmbedding.weight.data.copy_(torch.renorm(input=self.entityEmbedding.weight.detach().cpu(),
                                                            p=2,
                                                            dim=0,
                                                            maxnorm=1))
        self.relationEmbedding.weight.data.copy_(torch.renorm(input=self.relationEmbedding.weight.detach().cpu(),
                                                              p=2,
                                                              dim=0,
                                                              maxnorm=1))

    '''
    Return evaluation weights
    '''
    def retEvalWeights(self):
        return {"entityEmbed": self.entityEmbedding.weight.detach().cpu().numpy(),
                "relationEmbed": self.relationEmbedding.weight.detach().cpu().numpy(),
                "entityMapEmbed": self.entityMapEmbedding.weight.detach().cpu().numpy(),
                "relationMapEmbed": self.relationMapEmbedding.weight.detach().cpu().numpy()}

    def forward(self, posX, negX):
        size = posX.size()[0]

        # Calculate score
        posScore = self.scoreOp(posX)
        negScore = self.scoreOp(negX)

        return torch.sum(F.relu(input=posScore-negScore+self.margin)) / size


In [15]:
import pickle
import argparse
from torch.utils.data import DataLoader
from torch.autograd import Variable
from tensorboardX import SummaryWriter

In [27]:
dataloader = DataLoader(dataset,
                        batch_size=1024,
                        shuffle=True,
#                         num_workers=1,
                        drop_last=True)

In [17]:
class TransH(nn.Module):
    def __init__(self, entityNum, relationNum, embeddingDim, margin=3.0, L=2, C=1.0, eps=0.003):
        super(TransH, self).__init__()
        assert (L in [1, 2])
        self.model = "TransH"
        self.entnum = entityNum
        self.relnum = relationNum
        self.margin = margin
        self.L = L
        self.C = C
        self.eps = eps  #学习率

        self.entityEmbedding = nn.Embedding(num_embeddings=entityNum,
                                            embedding_dim=embeddingDim)
        self.relationEmbedding = nn.Embedding(num_embeddings=relationNum,
                                              embedding_dim=embeddingDim)
        self.relationHyper = nn.Embedding(num_embeddings=relationNum,
                                          embedding_dim=embeddingDim)
        self.distfn = nn.PairwiseDistance(L)

    '''
    Calculate the score:
    Step1 : Split the triple as head, relation and tail
    Step2 : Transform index tensor to embedding tensor
    Step3 : Project entity head and tail embedding to relation hyperplane
    Step4 : Calculate similarity score in relation hyperplane
    Step5 : Return the score
    '''
    def scoreOp(self, inputTriple):
        # Step1
        head, relation, tail = torch.chunk(inputTriple,
                                           chunks=3,
                                           dim=1)   #返回整型的tensor，可以作为Embedding的索引
        # Step2
        head = torch.squeeze(self.entityEmbedding(head), dim=1)   #shape(N,dim)
        relHyper = torch.squeeze(self.relationHyper(relation), dim=1)
        relation = torch.squeeze(self.relationEmbedding(relation), dim=1)
        tail = torch.squeeze(self.entityEmbedding(tail), dim=1)
        # Step3
        head = head - relHyper * torch.sum(head * relHyper, dim=1, keepdim=True) # *为逐元乘法，配合sum为点积
        tail = tail - relHyper * torch.sum(tail * relHyper, dim=1, keepdim=True)
        # Step4
        return self.distfn(head+relation, tail)

    '''
    Normalize relation hyper-plane embedding
    '''
    def normalizeEmbedding(self):
        hyperWeight = self.relationHyper.weight.detach().cpu().numpy()
        hyperWeight = hyperWeight / np.sqrt(np.sum(np.square(hyperWeight), axis=1, keepdims=True))  #长度化为1
        self.relationHyper.weight.data.copy_(torch.from_numpy(hyperWeight))

    def retEvalWeights(self):
        return {"entityEmbed": self.entityEmbedding.weight.detach().cpu().numpy(),
                "relationEmbed": self.relationEmbedding.weight.detach().cpu().numpy(),
                "hyperEmbed": self.relationHyper.weight.detach().cpu().numpy()}

    def forward(self, posX, negX):
        size = posX.size()[0]
        # Calculate score
        posScore = self.scoreOp(posX)
        negScore = self.scoreOp(negX)

        # Get margin ranking loss
        # max(posScore-negScore+margin, 0)
        # Use F.relu()
        marginLoss = torch.sum(F.relu(input=posScore-negScore+self.margin))
#         marginLoss = torch.sum(F.relu(input=posScore))
        entityLoss = torch.sum(F.relu(torch.norm(self.entityEmbedding.weight, p=2, dim=1, keepdim=False)-1)) #实体向量的长度，二范数大于一就有惩罚
        orthLoss = torch.sum(F.relu(torch.sum(self.relationHyper.weight * self.relationEmbedding.weight, dim=1, keepdim=False) / \
                                    torch.norm(self.relationEmbedding.weight, p=2, dim=1, keepdim=False) - self.eps ** 2))
        return  marginLoss/size +self.C*(entityLoss/self.entnum + orthLoss/self.relnum)
#         return marginLoss/size

In [18]:
entityNum=len(json.load(open(entityDictPath, "r",encoding='utf-8'))["stoi"])
relationNum=len(json.load(open(relationDictPath, "r",encoding='utf-8'))["stoi"])
transD=TransD(entityNum,relationNum,32,32)
# transH=TransH(entityNum,relationNum,36)
# transH.normalizeEmbedding()
transD.normalizeEmbedding()


In [114]:
transD.load_state_dict(torch.load('E:/project/xc_KG/EandR/code/process/data/transD32'))

<All keys matched successfully>

In [19]:
print(transD.parameters)


<bound method Module.parameters of TransD(
  (entityEmbedding): Embedding(90132, 32)
  (entityMapEmbedding): Embedding(90132, 32)
  (relationEmbedding): Embedding(424053, 32)
  (relationMapEmbedding): Embedding(424053, 32)
  (distfn): PairwiseDistance()
)>


In [28]:
def fit():
        EPOCHS = 20
        LR = 0.005
        OPTIMIZER = "Adam"
        if OPTIMIZER == "Adam":
            optimizer = torch.optim.Adam(transD.parameters(),
                                         weight_decay=0, #防止过拟合的权重衰减
                                         lr=LR)    #定义优化器，用的Adam
        else:
            print("ERROR : Optimizer %s is not supported."%OPTIMIZER)
            exit(1)

        # Training, GLOBALSTEP and GLOBALEPOCH are used for summary
        minLoss = float("inf")
        bestMR = float("inf")
        GLOBALSTEP = 0
        GLOBALEPOCH = 0
        for seed in range(100):
            print("INFO : Using seed %d" % seed)
#             dataloader = dataloader
            for epoch in range(EPOCHS):
                GLOBALEPOCH += 1
                STEP = 0
                print("="*20+"EPOCHS(%d/%d)"%(epoch+1, EPOCHS)+"="*20)
                for posX, negX in dataloader:
                    # Allocate tensor to devices
                    posX = Variable(torch.LongTensor(posX))
                    negX = Variable(torch.LongTensor(negX))

            
                    # Calculate the loss from the model
                    loss = transD(posX, negX)             #此处有模型信息
                    lossVal = loss.item()
#                     print(lossVal)

                    # Calculate the gradient and step down
                    optimizer.zero_grad()  #pytorch梯度计算后不会自动清零，因此每次sgd要手动清零
                    loss.backward()
                    optimizer.step()
                    print(transD.relationEmbedding)

                    # Print infomation and add to summary
                    if minLoss > lossVal:   #一定执行
                        minLoss = lossVal
                    print("[TRAIN-EPOCH(%d/%d)-STEP(%d)]Loss:%.4f, minLoss:%.4f"%(epoch+1, EPOCHS, STEP, lossVal, minLoss))
                    STEP += 1
                    GLOBALSTEP += 1
#                     sumWriter.add_scalar('train/loss', lossVal, global_step=GLOBALSTEP)
#                 if GLOBALEPOCH % self.args.lrdecayepoch == 0:
#                     adjust_learning_rate(optimizer, decay=self.args.lrdecay)
#                 if GLOBALEPOCH % self.args.evalepoch == 0:
#                     MR = evaluation.MREvaluation(evalloader=self.evalloader,
#                                                  model=self.args.modelname,
#                                                  simMeasure=args.simmeasure,
#                                                  **self.model.retEvalWeights())
#                     sumWriter.add_scalar('train/eval', MR, global_step=GLOBALEPOCH)
#                     print("[EVALUATION-EPOCH(%d/%d)]Measure method %s, eval %.4f"% \
#                           (epoch+1, EPOCHS, self.args.evalmethod, MR))
#                     # Save the model if new MR is better
#                     if MR < bestMR:
#                         bestMR = MR
#                         self.saveModel()
#                         self.dumpEmbedding()

In [29]:
fit()

INFO : Using seed 0
====================EPOCHS(1/20)====================
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(0)]Loss:1.2781, minLoss:1.2781
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(1)]Loss:1.1885, minLoss:1.1885
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(2)]Loss:1.1979, minLoss:1.1885
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(3)]Loss:1.1648, minLoss:1.1648
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(4)]Loss:1.4104, minLoss:1.1648
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(5)]Loss:1.0484, minLoss:1.0484
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(6)]Loss:1.1322, minLoss:1.0484
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(7)]Loss:1.2232, minLoss:1.0484
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(8)]Loss:0.9706, minLoss:0.9706
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(9)]Loss:1.2330, minLoss:0.9706
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(10)]Loss:1.1759, minLoss:0.9706
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(11)]Loss:1.0494, minLoss:0.9706
E

[TRAIN-EPOCH(1/20)-STEP(207)]Loss:1.0766, minLoss:0.8989
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(208)]Loss:1.1036, minLoss:0.8989
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(209)]Loss:1.0122, minLoss:0.8989
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(210)]Loss:1.0770, minLoss:0.8989
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(211)]Loss:1.0781, minLoss:0.8989
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(212)]Loss:0.9920, minLoss:0.8989
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(213)]Loss:1.0273, minLoss:0.8989
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(214)]Loss:1.0388, minLoss:0.8989
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(215)]Loss:1.0300, minLoss:0.8989
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(216)]Loss:1.0909, minLoss:0.8989
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(217)]Loss:1.0019, minLoss:0.8989
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(218)]Loss:1.0624, minLoss:0.8989
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(219)]Loss:1.0348, minLoss:0.

[TRAIN-EPOCH(1/20)-STEP(413)]Loss:1.0738, minLoss:0.8922
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(414)]Loss:0.9551, minLoss:0.8922
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(415)]Loss:0.9862, minLoss:0.8922
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(416)]Loss:1.0996, minLoss:0.8922
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(417)]Loss:0.9249, minLoss:0.8922
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(418)]Loss:0.9980, minLoss:0.8922
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(419)]Loss:1.0065, minLoss:0.8922
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(420)]Loss:0.9257, minLoss:0.8922
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(421)]Loss:0.9782, minLoss:0.8922
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(422)]Loss:0.9495, minLoss:0.8922
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(423)]Loss:1.1057, minLoss:0.8922
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(424)]Loss:1.0118, minLoss:0.8922
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(425)]Loss:0.9657, minLoss:0.

[TRAIN-EPOCH(1/20)-STEP(619)]Loss:1.0596, minLoss:0.8622
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(620)]Loss:0.9728, minLoss:0.8622
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(621)]Loss:0.9365, minLoss:0.8622
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(622)]Loss:0.9129, minLoss:0.8622
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(623)]Loss:1.0594, minLoss:0.8622
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(624)]Loss:0.9916, minLoss:0.8622
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(625)]Loss:0.9793, minLoss:0.8622
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(626)]Loss:0.9422, minLoss:0.8622
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(627)]Loss:1.0322, minLoss:0.8622
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(628)]Loss:1.0019, minLoss:0.8622
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(629)]Loss:0.8880, minLoss:0.8622
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(630)]Loss:1.0315, minLoss:0.8622
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(631)]Loss:0.9422, minLoss:0.

[TRAIN-EPOCH(1/20)-STEP(825)]Loss:0.9944, minLoss:0.8354
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(826)]Loss:0.8959, minLoss:0.8354
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(827)]Loss:0.9323, minLoss:0.8354
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(828)]Loss:0.9673, minLoss:0.8354
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(829)]Loss:1.0242, minLoss:0.8354
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(830)]Loss:0.8993, minLoss:0.8354
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(831)]Loss:1.0109, minLoss:0.8354
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(832)]Loss:0.9922, minLoss:0.8354
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(833)]Loss:0.9952, minLoss:0.8354
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(834)]Loss:0.9926, minLoss:0.8354
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(835)]Loss:0.9758, minLoss:0.8354
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(836)]Loss:0.8938, minLoss:0.8354
Embedding(424053, 32)
[TRAIN-EPOCH(1/20)-STEP(837)]Loss:1.0260, minLoss:0.

...

[TRAIN-EPOCH(6/20)-STEP(992)]Loss:0.2655, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(993)]Loss:0.2571, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(994)]Loss:0.2182, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(995)]Loss:0.2368, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(996)]Loss:0.2177, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(997)]Loss:0.2229, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(998)]Loss:0.1973, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(999)]Loss:0.2220, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1000)]Loss:0.2299, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1001)]Loss:0.1901, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1002)]Loss:0.2114, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1003)]Loss:0.2270, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1004)]Loss:0.2048, minLo

[TRAIN-EPOCH(6/20)-STEP(1196)]Loss:0.2207, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1197)]Loss:0.2064, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1198)]Loss:0.2292, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1199)]Loss:0.2276, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1200)]Loss:0.2161, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1201)]Loss:0.1961, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1202)]Loss:0.1938, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1203)]Loss:0.2153, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1204)]Loss:0.2079, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1205)]Loss:0.2482, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1206)]Loss:0.2698, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1207)]Loss:0.2299, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1208)]Loss:0.215

[TRAIN-EPOCH(6/20)-STEP(1400)]Loss:0.2401, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1401)]Loss:0.2332, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1402)]Loss:0.2098, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1403)]Loss:0.2666, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1404)]Loss:0.2280, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1405)]Loss:0.2680, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1406)]Loss:0.2314, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1407)]Loss:0.2267, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1408)]Loss:0.2722, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1409)]Loss:0.2344, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1410)]Loss:0.2605, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1411)]Loss:0.2148, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1412)]Loss:0.267

[TRAIN-EPOCH(6/20)-STEP(1604)]Loss:0.2787, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1605)]Loss:0.2142, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1606)]Loss:0.2233, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1607)]Loss:0.2241, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1608)]Loss:0.2514, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1609)]Loss:0.2251, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1610)]Loss:0.1954, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1611)]Loss:0.2682, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1612)]Loss:0.1728, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1613)]Loss:0.2233, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1614)]Loss:0.1858, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1615)]Loss:0.2594, minLoss:0.1551
Embedding(424053, 32)
[TRAIN-EPOCH(6/20)-STEP(1616)]Loss:0.231

KeyboardInterrupt: 

In [30]:
# transD.normalizeEmbedding()
torch.save(transD.state_dict(),'E:/project/xc_KG/EandR/code/process/data/transD32_C-dayu2')